In [9]:
import tkinter as tk
from tkinter import messagebox, ttk
import mysql.connector
from tkinter import simpledialog
from datetime import datetime

# Connect to the MySQL database
def connect_db():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Amrzain#22",
            database="Delivery_System"
        )
        return conn
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")
        return None

# Fetch and display customer data
def show_customers():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Customer")
    customers = cursor.fetchall()
    cursor.close()
    conn.close()
    
    top = tk.Toplevel()
    top.title("Customers")
    tree = ttk.Treeview(top, columns=("ID", "Name", "Address"), show="headings")
    tree.heading("ID", text="ID")
    tree.heading("Name", text="Name")
    tree.heading("Address", text="Address")
    tree.grid(row=0, column=0, padx=10, pady=10)
    
    for customer in customers:
        tree.insert("", "end", values=customer)

# Fetch and display restaurant data
def show_restaurants():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Restaurant")
    restaurants = cursor.fetchall()
    cursor.close()
    conn.close()
    
    top = tk.Toplevel()
    top.title("Restaurants")
    tree = ttk.Treeview(top, columns=("TC", "Name", "Address"), show="headings")
    tree.heading("TC", text="TC")
    tree.heading("Name", text="Name")
    tree.heading("Address", text="Address")
    tree.grid(row=0, column=0, padx=10, pady=10)
    
    for restaurant in restaurants:
        tree.insert("", "end", values=restaurant)

# Fetch and display menu data
def show_menus():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT m.Item_ID, m.IName, m.Price, r.RName FROM Menus m JOIN Restaurant r ON m.RES_TC = r.TC")
    menus = cursor.fetchall()
    cursor.close()
    conn.close()
    
    top = tk.Toplevel()
    top.title("Menus")
    tree = ttk.Treeview(top, columns=("Item_ID", "Item Name", "Price", "Restaurant"), show="headings")
    tree.heading("Item_ID", text="Item ID")
    tree.heading("Item Name", text="Item Name")
    tree.heading("Price", text="Price")
    tree.heading("Restaurant", text="Restaurant")
    tree.grid(row=0, column=0, padx=10, pady=10)
    
    for menu in menus:
        tree.insert("", "end", values=menu)

# Fetch and display previous orders
def show_previous_orders():
    conn = connect_db()
    if conn is None:
        return

    cursor = conn.cursor()
    try:
        cursor.execute("""
        SELECT * FROM `order`
        """)
        orders = cursor.fetchall()
        if not orders:
            messagebox.showinfo("No Orders", "No previous orders found.")
            return
        # Display the orders in a Treeview (as previously described)
    except mysql.connector.Error as err:
        messagebox.showerror("Database Error", f"Error: {err}")
    finally:
        cursor.close()
        conn.close()
    top = tk.Toplevel()
    top.title("Previous Orders")

    # Create the Treeview to display order details
    tree = ttk.Treeview(top, columns=("Order_ID", "Item Name", "Cost", "Payment", "Date", "Customer Name", "Customer Address", "Delivery Name", "Vehicle"), show="headings")
    tree.heading("Order_ID", text="Order ID")
    tree.heading("Item Name", text="Item Name")
    tree.heading("Cost", text="Cost")
    tree.heading("Payment", text="Payment")
    tree.heading("Date", text="Date")
    tree.heading("Customer Name", text="Customer Name")
    tree.heading("Customer Address", text="Customer Address")
    tree.heading("Delivery Name", text="Delivery Name")
    tree.heading("Vehicle", text="Vehicle")
    tree.grid(row=0, column=0, padx=10, pady=10)

    # Loop through the orders and insert them into the Treeview
    for order in orders:
        tree.insert("", "end", values=order)


# Function to choose a restaurant using a dropdown (ComboBox)
def choose_restaurant():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT TC, RName FROM Restaurant")
    restaurants = cursor.fetchall()
    cursor.close()
    conn.close()

    restaurant_names = [r[1] for r in restaurants]
    
    def on_select_restaurant(event):
        restaurant_name = restaurant_combobox.get()
        restaurant_id = next(r[0] for r in restaurants if r[1] == restaurant_name)
        show_menu(restaurant_id)

    # Create a popup window for restaurant selection
    top = tk.Toplevel()
    top.title("Select Restaurant")

    tk.Label(top, text="Select a Restaurant:").pack(padx=10, pady=5)
    restaurant_combobox = ttk.Combobox(top, values=restaurant_names)
    restaurant_combobox.pack(padx=10, pady=10)
    restaurant_combobox.bind("<<ComboboxSelected>>", on_select_restaurant)

# Function to show menu items for the selected restaurant with checkboxes
def show_menu(restaurant_id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT Item_ID, IName, Price FROM Menus WHERE RES_TC = %s", (restaurant_id,))
    menu_items = cursor.fetchall()
    cursor.close()
    conn.close()

    def on_item_selected():
        items_id = [item[0] for item in menu_items if item_var[item[0]].get()]
        total_cost = sum(item[2] for item in menu_items if item_var[item[0]].get())
        choose_payment(items_id, total_cost)

    # Create a popup window to select menu items
    top = tk.Toplevel()
    top.title("Select Items")

    tk.Label(top, text="Select Items:").pack(padx=10, pady=5)

    item_var = {}
    for item in menu_items:
        var = tk.IntVar()
        item_var[item[0]] = var
        tk.Checkbutton(top, text=f"{item[1]} - ${item[2]}", variable=var).pack(padx=10, anchor="w")
    
    tk.Button(top, text="Next", command=on_item_selected).pack(padx=10, pady=10)


# Function to choose payment method
def choose_payment(items_id, total_cost):
    payment_method = simpledialog.askstring("Payment", "Enter Payment Method (CASH or VISA):")

    if payment_method in ["CASH", "VISA","cash","visa"]:
        choose_delivery(items_id, total_cost,payment_method)
    else:
        messagebox.showerror("Error", "Invalid payment method. Please choose 'CASH' or 'VISA'.")



# Function to choose delivery using a dropdown (ComboBox)
def choose_delivery(items_id, total_cost, payment_method):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT SSN, DName FROM Delivery")
    deliveries = cursor.fetchall()
    cursor.close()
    conn.close()

    delivery_names = [d[1] for d in deliveries]

    def on_select_delivery(event):
        delivery_name = delivery_combobox.get()
        delivery_ssn = next(d[0] for d in deliveries if d[1] == delivery_name)
        show_order_summary(items_id, total_cost, delivery_ssn,payment_method)

    # Create a popup window for delivery selection
    top = tk.Toplevel()
    top.title("Select Delivery")

    tk.Label(top, text="Select a Delivery Person:").pack(padx=10, pady=5)
    delivery_combobox = ttk.Combobox(top, values=delivery_names)
    delivery_combobox.pack(padx=10, pady=10)
    delivery_combobox.bind("<<ComboboxSelected>>", on_select_delivery)

# Function to show the order summary (without inserting into the database yet)
def show_order_summary(items_id, total_cost, delivery_ssn,payment_method):
    # Display the order details in a confirmation message
    order_details = f"Order Summary:\n\nItems: {', '.join(map(str, items_id))}\nTotal Cost: ${total_cost}\nDelivery Person: {delivery_ssn}\nPayment method: {payment_method}"
    
    confirm = messagebox.askyesno("Order Summary", f"{order_details}\n\nDo you want to proceed with the order?")
    
    if confirm:
        # If user confirms, show customer details entry
        enter_customer_details(items_id, total_cost, delivery_ssn,payment_method)
    else:
        messagebox.showinfo("Order Cancelled", "Order was not placed.")



# Function to enter customer details
def enter_customer_details(items_id, total_cost, delivery_ssn, payment_method):
    customer_name = simpledialog.askstring("Customer Name", "Enter your name:")
    customer_address = simpledialog.askstring("Customer Address", "Enter your address:")

    if customer_name and customer_address:
        # Insert customer details into the Customer table
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("""
            INSERT INTO Customer (CName, CAddress)
            VALUES (%s, %s)
        """, (customer_name, customer_address))
        conn.commit()

        # Get the last inserted customer ID
        cursor.execute("SELECT LAST_INSERT_ID()")
        customer_id = cursor.fetchone()[0]

        cursor.close()
        conn.close()

        # Now that we have the customer ID, proceed with confirming the order
        confirm_order(items_id, total_cost, delivery_ssn, customer_id, customer_name, customer_address, payment_method)
    else:
        messagebox.showerror("Error", "Customer name and address are required.")


# Function to confirm and display order details
# Function to confirm and display order details
def confirm_order(items_id, total_cost, delivery_ssn, customer_id, customer_name, customer_address, payment_method):
    try:
        conn = connect_db()
        cursor = conn.cursor()

        # Insert the order into the order table
        cursor.execute("""
        INSERT INTO `order` (Statue, ODate, Cost, payment, DSSN, CID)
         VALUES (%s, NOW(), %s, %s, %s, %s)
        """, ('pending', total_cost, payment_method, delivery_ssn, customer_id))
        
        # Get the last inserted order ID (immediately after insertion)
        cursor.execute("SELECT LAST_INSERT_ID()")
        order_id = cursor.fetchone()[0]
        conn.commit()  # Commit the order first to avoid the foreign key constraint issue

        # Check if items_id is not empty
        if not items_id:
            raise ValueError("No items selected for the order.")

        # Insert each item into the order_items table
        for item_id in items_id:
            # Check if item_id is valid
            if item_id is None or item_id <= 0:
                raise ValueError(f"Invalid item ID: {item_id}")

            cursor.execute("""
                INSERT INTO order_items (order_ID, item_ID)
                VALUES (%s, %s)
            """, (order_id, item_id))

        conn.commit()  # Commit the items after inserting them

        cursor.close()
        conn.close()

        messagebox.showinfo("Order Confirmation", "Your order has been placed successfully!")

    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"MySQL error occurred: {err}")
        print(f"MySQL error occurred: {err}")
    except ValueError as err:
        messagebox.showerror("Error", f"Invalid input: {err}")
    except Exception as err:
        messagebox.showerror("Error", f"An unexpected error occurred: {err}")




# Set up the main window
root = tk.Tk()
root.title("Delivery System")

# Add buttons for each section
tk.Button(root, text="Customers", command=show_customers).grid(row=0, column=0, padx=10, pady=10)
tk.Button(root, text="Restaurants", command=show_restaurants).grid(row=0, column=2, padx=10, pady=10)
tk.Button(root, text="Menus", command=show_menus).grid(row=1, column=0, padx=10, pady=10)
tk.Button(root, text="Previous Orders", command=show_previous_orders).grid(row=1, column=2, padx=10, pady=10)
tk.Button(root, text="New Order", command=choose_restaurant).grid(row=2, column=1, padx=10, pady=10)

# Run the main event loop
root.mainloop()